# db Setting

In [2]:
import sqlite3, pathlib

path = "/home/work/CoreIQ/Ruah/backend/storage/pps_rag.db"
schema_path = "/home/work/CoreIQ/Ruah/backend/storage/schema.sql"

sql = pathlib.Path(schema_path).read_text(encoding="utf-8")
con = sqlite3.connect(path)
con.execute("PRAGMA foreign_keys=ON;")
con.executescript(sql)
con.commit()
con.close()


def get_db():
    conn = sqlite3.connect(path)
    conn.row_factory = sqlite3.Row
    return conn

# 유저

In [4]:
import sqlite3
import hashlib
from typing import Optional


def hash_password(plain_password: str) -> str:
    """Return a SHA-256 hex digest for the given plain password.
    Consider switching to bcrypt/argon2 in production environments.
    """
    return hashlib.sha256(plain_password.encode("utf-8")).hexdigest()


def create_user(
    username: str,
    name: str,
    password: str,
    department: str,
    position: str,
    role: str = "user",
    pfp_filename: Optional[str] = None,
    bio: str = "",
    daily_message_limit: Optional[int] = None,
    suspended: int = 0,
    security_level: int = 3,
) -> int:
    """Insert a new user row into the users table and return the inserted row id.

    Required: username, name, password, department, position
    Optional: role, pfp_filename, bio, daily_message_limit, suspended, security_level
    """
    hashed = hash_password(password)

    conn = get_db()
    try:
        cursor = conn.cursor()
        cursor.execute(
            """
            INSERT INTO users (
                role, username, name, password, department, position,
                pfp_filename, bio, daily_message_limit, suspended, security_level
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            (
                role,
                username,
                name,
                hashed,
                department,
                position,
                pfp_filename,
                bio,
                daily_message_limit,
                suspended,
                security_level,
            ),
        )
        conn.commit()
        return cursor.lastrowid
    except sqlite3.IntegrityError as exc:
        # Likely a UNIQUE constraint failure on username
        raise ValueError(f"Failed to create user: {exc}") from exc
    finally:
        conn.close()

In [7]:
# 사용 예시 (필요 시 값 변경 후 실행)
new_user_id = create_user(
    role="user",
    username="gijung123",
    name="조기정",
    password="1234",
    department="AI 연구소",
    position="선임연구원",
    security_level=2,
)
print("inserted user id:", new_user_id)


inserted user id: 3


In [4]:
def delete_user_by_id(user_id: int) -> int:
    """
    users 테이블에서 해당 ID 삭제.
    관련 user_sessions도 함께 삭제.
    반환: 삭제된 사용자 개수(0 또는 1)
    """
    conn = get_db()
    try:
        cur = conn.cursor()
        # 매핑 먼저 정리
        cur.execute("DELETE FROM user_sessions WHERE user_id = ?", (user_id,))
        # 사용자 삭제
        cur.execute("DELETE FROM users WHERE id = ?", (user_id,))
        deleted = cur.rowcount
        conn.commit()
        return deleted
    finally:
        conn.close()

In [ ]:
delete_user_by_id(2)

# System prompt
## create prompt

In [6]:
import json
from typing import Optional, List, Dict, Any

def create_system_prompt_template(
    name: str,
    category: str,       # "qa" | "doc_gen" | "summary"
    content: str,
    required_vars: Optional[List[str]] = None,   # ["document_title","author"] 등
    is_default: bool = False,
    is_active: bool = True,
    variable_specs: Optional[List[Dict[str, Any]]] = None,
) -> int:
    """
    system_prompt_template를 생성합니다.
    - category가 'doc_gen'이면 변수(system_prompt_variables)와 매핑(prompt_mapping)도 함께 생성합니다.
    - variable_specs 형식 예:
      [
        {"type": "text", "key": "제목", "value": None, "description": "예: 일본 출장 계획서"},
        {"type": "datetime", "key": "마감일", "description": "예: 2025-08-12"},
      ]
    - variable_specs가 없고 required_vars만 전달되면, doc_gen의 경우 'text' 타입으로 자동 생성합니다.
    """
    # doc_gen 변수 이름 목록 결정
    if category == "doc_gen":
        if not variable_specs and not required_vars:
            raise ValueError("doc_gen 카테고리는 variable_specs 또는 required_vars가 필요합니다.")
        if not variable_specs and required_vars:
            # required_vars만 있으면 text 타입으로 자동 구성
            variable_specs = [{"type": "text", "key": k} for k in required_vars]
        var_names = [v["key"] for v in variable_specs]  # 템플릿 required_vars에 저장될 키 목록
    else:
        var_names = required_vars or []

    conn = get_db()
    try:
        cur = conn.cursor()

        # 1) 템플릿 생성
        cur.execute(
            """
            INSERT INTO system_prompt_template
              (name, category, content, required_vars, is_default, is_active)
            VALUES (?, ?, ?, ?, ?, ?)
            """,
            (
                name,
                category,
                content,
                json.dumps(var_names, ensure_ascii=False),
                1 if is_default else 0,
                1 if is_active else 0,
            ),
        )
        template_id = cur.lastrowid

        # 2) doc_gen이면 변수/매핑 생성
        if category == "doc_gen":
            # 변수 생성
            var_ids: List[int] = []
            for spec in variable_specs or []:
                vtype = spec.get("type", "text")
                key = spec["key"]
                value = spec.get("value")
                description = spec.get("description", "")
                cur.execute(
                    """
                    INSERT INTO system_prompt_variables (type, key, value, description)
                    VALUES (?, ?, ?, ?)
                    """,
                    (vtype, key, value, description),
                )
                var_ids.append(cur.lastrowid)

            # 매핑 생성
            for vid in var_ids:
                cur.execute(
                    """
                    INSERT INTO prompt_mapping (template_id, variable_id)
                    VALUES (?, ?)
                    """,
                    (template_id, vid),
                )

        conn.commit()
        return template_id
    except Exception:
        conn.rollback()
        raise
    finally:
        conn.close()




In [36]:
tid = create_system_prompt_template(
    name="채용 공고",
    category="doc_gen",
    content="""
당신은 채용 공고 문서를 작성하는 AI 어시스턴트입니다.  
아래 제공된 변수 값을 참고하여 회사와 채용 포지션 정보를 명확하고 매력적으로 문서 형식으로 작성하세요.

작성 규칙:
1. 문서는 직무명, 회사 소개, 담당 업무, 자격 요건, 우대 사항, 근무 조건, 지원 방법 순으로 구성
2. 중요 정보가 돋보이도록 불릿 포인트와 명확한 문장 사용
3. 불필요한 수식어나 반복 표현은 제거하며, 지원자 입장에서 이해하기 쉽게 작성
4. 제공된 변수들을 자연스럽게 연결하여 문장 작성
5. 날짜는 YYYY-MM-DD 형식으로 표기
""",
    is_default=True,
    variable_specs=[
        {"type": "text", "key": "직무명", "description": "채용하는 직무명 (예: 소프트웨어 엔지니어)"},
        {"type": "text", "key": "회사소개", "description": "회사 개요 및 비전 소개"},
        {"type": "text", "key": "담당업무", "description": "직무별 주요 업무 내용"},
        {"type": "text", "key": "자격요건", "description": "필수 자격 및 능력 조건"},
        {"type": "text", "key": "우대사항", "description": "우대하는 경험이나 기술 (선택 사항)"},
        {"type": "text", "key": "근무조건", "description": "근무지, 근무 형태, 근무 시간 등 근무 관련 사항"},
        {"type": "datetime", "key": "지원마감일", "description": "채용 지원 마감일"},
        {"type": "text", "key": "지원방법", "description": "지원 절차나 연락처 등 안내"}
    ],
)


In [ ]:
# 사용 예시:
tid = create_system_prompt_template(
    name="기본 QA Prompt",
    category="qa",
    content="""당신은 친절하고 이해하기 쉬운 설명을 제공하는 AI 어시스턴트입니다.
사용자의 질문이나 요청에 대해 정확하고 간결하게, 그리고 가능하다면 추가적인 배경 지식과 예시를 곁들여 답변하세요.
어려운 용어나 개념이 나올 경우, 초보자도 이해할 수 있도록 쉽게 풀어 설명하고, 필요하다면 목록·표·코드 블록 등을 활용하세요.
대화는 항상 존중과 긍정적인 어조를 유지하며, 사용자의 의도와 목표를 먼저 확인한 뒤에 답변을 구성합니다.

응답 스타일 가이드:

핵심 정보 먼저 제시

불필요하게 긴 문장 대신 명확한 구조

예시, 정의, 비교 설명 활용

질문이 모호하면 추가 질문으로 의도 명확히 하기

사용자가 원할 경우 심화 정보 제공""",
    required_vars=None,
    is_default=True,
    is_active=True,
)
print("inserted template id:", tid)

### delete prompt

In [27]:

def delete_system_prompt_template_by_id(template_id: int) -> int:
    """
    system_prompt_template에서 해당 ID 삭제.
    관련 prompt_mapping도 함께 삭제.
    반환: 삭제된 템플릿 개수(0 또는 1)
    """
    conn = get_db()
    try:
        cur = conn.cursor()
        # 매핑 먼저 정리
        cur.execute("DELETE FROM prompt_mapping WHERE template_id = ?", (template_id,))
        # 템플릿 삭제
        cur.execute("DELETE FROM system_prompt_template WHERE id = ?", (template_id,))
        deleted = cur.rowcount
        conn.commit()
        return deleted
    finally:
        conn.close()

In [ ]:
delete_system_prompt_template_by_id(3)

# Save LLM Models

In [1]:
import sqlite3
from typing import Optional

def create_llm_model(
    provider: str,
    name: str,
    category: str,        # "qa" | "doc_gen" | "summary"
    type_: str = "base",  # "base" | "lora" | "full"
    revision: Optional[int] = None,
    model_path: Optional[str] = None,
    is_default: bool = False,
    is_active: bool = True,
    trained_at: Optional[str] = None,  # "YYYY-MM-DD HH:MM:SS" 또는 None
) -> int:
    if category not in ("qa", "doc_gen", "summary"):
        raise ValueError("category must be one of: qa, doc_gen, summary")
    if type_ not in ("base", "lora", "full"):
        raise ValueError("type must be one of: base, lora, full")

    conn = get_db()
    try:
        cur = conn.cursor()
        cur.execute(
            """
            INSERT INTO llm_models
              (provider, name, revision, model_path, category, type, is_default, is_active, trained_at)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            (
                provider,
                name,
                revision,
                model_path,
                category,
                type_,
                1 if is_default else 0,
                1 if is_active else 0,
                trained_at,
            ),
        )
        conn.commit()
        return cur.lastrowid
    except sqlite3.IntegrityError as e:
        # name은 UNIQUE, 카테고리별 is_default=1은 트리거+부분 유니크 인덱스
        raise ValueError(f"Failed to create llm_model: {e}") from e
    finally:
        conn.close()

In [ ]:
# 사용 예시:
mid = create_llm_model(
    provider="huggingface",
    name="gpt_oss_20b_doc_gen",
    category="doc_gen",
    type_="base",
    model_path="/home/work/CoreIQ/gpu_use/KT_sever/local_gpt_oss_20b",
    is_active=True,
    is_default=True,   # 같은 카테고리 기존 기본값은 자동 해제됨
)
print("inserted model id:", mid)


"/home/work/CoreIQ/gpu_use/KT_sever/local_Qwen2.5-VL-7B-Instruct"
"/home/work/CoreIQ/gpu_use/KT_sever/local_Qwen2.5-7B-Instruct"
"/home/work/CoreIQ/gpu_use/KT_sever/local_gpt_oss_20b"

inserted model id: 13


'/home/work/CoreIQ/gpu_use/KT_sever/local_gpt_oss_20b'

# Change Default LLM Model

In [5]:
def set_default_llm_model(name: str, category: str ) -> bool:
    """
    - name으로 대상 모델 조회
    - (옵션) category가 주어지면 대상 모델의 category와 일치 검증
    - 같은 category에서 대상만 is_default=1, 나머지는 0으로 원샷 토글
    - 비활성(is_active=0) 모델은 기본값 불가
    """
    con = get_db()
    try:
        cur = con.cursor()
        cur.execute("SELECT id, category, is_active FROM llm_models WHERE name=?", (name,))
        row = cur.fetchone()
        print(row)
        if not row:
            print(f"[WARN] LLM model not found: name={name}")
            return False

        model_id = row["id"]
        model_category = row["category"]
        is_active = bool(row["is_active"])

        if category and category != model_category:
            raise ValueError(f"category mismatch: expected '{category}', got '{model_category}' for model '{name}'")
        if not is_active:
            raise ValueError(f"cannot set inactive model as default: name={name}")

        # 같은 카테고리에서 대상만 1, 나머지는 0
        cur.execute(
            """
            UPDATE llm_models
            SET is_default = CASE WHEN id = ? THEN 1 ELSE 0 END
            WHERE category = ?
            """,
            (model_id, model_category),
        )
        con.commit()
        print(f"[OK] Default LLM set: category={model_category}, name={name}")
        return cur.rowcount > 0
    except sqlite3.IntegrityError as e:
        con.rollback()
        raise

In [8]:
set_default_llm_model("qwen_2.5_7b_instruct", category="qa")
#set_default_llm_model("gpt_oss_20b", category="qa")


[OK] Default LLM set: category=qa, name=qwen_2.5_7b_instruct


True

In [27]:

def list_llm_models(category: str):
    """
    카테고리별 모델/기본값 상태 확인용
    """
    con = get_db()
    try:
        cur = con.cursor()
        if category:
            cur.execute(
                "SELECT id, provider, name, category, is_default, is_active FROM llm_models WHERE category=? ORDER BY id",
                (category,),
            )
        else:
            cur.execute(
                "SELECT id, provider, name, category, is_default, is_active FROM llm_models ORDER BY category, id"
            )
        rows = cur.fetchall()
        for r in rows:
            print(f"{r['id']:>3} | {r['category']:<8} | {r['provider']:<11} | {r['name']:<24} | default={r['is_default']} | active={r['is_active']}")
        return rows
    finally:
        con.close()

In [28]:
list_llm_models("qa")

  1 | qa       | huggingface | qwen-2.5-vl-7b-instruct  | default=0 | active=1
  2 | qa       | huggingface | qwen-2.5-7b-instruct     | default=1 | active=1
  3 | qa       | huggingface | gpt_oss_20b              | default=0 | active=1


In [33]:
import sqlite3, os

DB = "/home/work/CoreIQ-RA/backend/storage/pps_rag.db"
con = sqlite3.connect(DB); con.row_factory = sqlite3.Row
cur = con.cursor()

print("db exists:", os.path.exists(DB))
print("rows:", cur.execute("SELECT COUNT(*) FROM llm_models").fetchone()[0])

for r in cur.execute("SELECT id, provider, name, length(name) AS L, hex(name) AS HX FROM llm_models"):
    print(r["id"], r["provider"], repr(r["name"]), r["L"], r["HX"])

target = "gpt-oss-20b"  # 실제 찾을 이름
print("target:", repr(target), len(target), target.encode().hex())

print(cur.execute("SELECT id,name FROM llm_models WHERE name = ?", (target,)).fetchone())
print(cur.execute("SELECT id,name FROM llm_models WHERE TRIM(name)=TRIM(?)", (target,)).fetchone())
print(cur.execute("SELECT id,name FROM llm_models WHERE name = ? COLLATE NOCASE", (target,)).fetchone())

db exists: True
rows: 3
1 huggingface 'qwen-2.5-vl-7b-instruct' 23 7177656E2D322E352D766C2D37622D696E737472756374
2 huggingface 'qwen-2.5-7b-instruct' 20 7177656E2D322E352D37622D696E737472756374
3 huggingface 'gpt_oss_20b' 11 6770745F6F73735F323062
target: 'gpt-oss-20b' 11 6770742d6f73732d323062
None
None
None
